In [ ]:
import os, glob, json, csv, subprocess, sys, re
from urllib.request import urlopen
from itertools import chain
import urllib.request
from pprint import pprint
from bs4 import BeautifulSoup
from git import *
from subprocess import Popen, PIPE
from os import path
from collections import Counter
import pandas as pd
import plotly.graph_objs as go
import plotly
from plotly.offline import plot, init_notebook_mode, iplot
import itertools

%matplotlib inline

# Defining repository

In [ ]:
userhome = os.path.expanduser('~')

In [ ]:
repository = userhome + r'/different-diff/dataset/hbase/'

In [ ]:
analyze_dir = userhome + r'/different-diff/analyze/analyze_hbase/'

# Defining git command function

In [ ]:
def execute_command(cmd, work_dir):
    """Executes a shell command in a subprocess, waiting until it has completed.
    :param cmd: Command to execute.
    :param work_dir: Working directory path.
    """
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

In [ ]:
def execute_shell_command(buglink, id1, id2, cmd, work_dir):
    #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return buglink, id1, id2, out, error
    pipe.wait()

# Finding log messages containing bug link

In [ ]:
words = ['bug', 'fix', 'defect', 'patch']
logs = []
for n, word in enumerate(words):
    git_cmd = "git log --all --grep='" + word + "' --oneline"
    log = (str(execute_command(git_cmd, repository)).replace("b'",'').replace('b"','').replace('(','',1).replace("\\'","'").split("\\n"))[:-1]

    logs.append(log)

com_logs = [item for sublist in logs for item in sublist]
com_logs

In [ ]:
len(com_logs)

In [ ]:
commit = []
for xx in range(0,len(com_logs)):
    tmp = []
    comm = com_logs[xx].split()
    word = ' '.join(comm[1:])
    tmp.extend([comm[0],word])
    commit.append(tmp)

for item in commit:
    print (item)

In [ ]:
commitbugs = []
buglinks = []

for xx in range(0,len(com_logs)):
    tmp = []
    comm = com_logs[xx].split()
    splitres = (str(comm[1:]))
    letter = re.findall(r"HBASE+[-][0-9]+", splitres)
    if letter != []:
        commitbugs.extend(letter)
    else:
        pass

commitbugs = set(commitbugs)
for p,q in enumerate(commitbugs):
    buglinks.append(q)
buglinks = sorted(buglinks, reverse=True)
print (buglinks)
print ("\nNumber of bug id: " + str(len(buglinks)))

In [ ]:
errorlinks = []
id_type = []
for a,b in enumerate(buglinks):
    link = "https://issues.apache.org/jira/browse/" + b
    sys.stdout.write("\r%i " %(a+1) + "Extracting: " + b)
    sys.stdout.flush()
    try:
        page = urllib.request.urlopen(link)
        soup = BeautifulSoup(page, 'html.parser')
        types = soup.find('span', attrs={'id':'type-val'}).text.replace("\n",'').replace(" ",'').split(",")
        types = sorted(types)
        types.insert(0, b)
        id_type.append(types)
    except:
        errorlinks.append(b)

print("\nExtraction has been completed.")

In [ ]:
type_of_id = id_type
errorlinks

In [ ]:
error_links = []
for a,b in enumerate(errorlinks):
    link = "https://issues.apache.org/jira/browse/" + b
    sys.stdout.write("\r%i " %(a+1) + "Extracting: " + b)
    sys.stdout.flush()
    try:
        page = urllib.request.urlopen(link)
        soup = BeautifulSoup(page, 'html.parser')
        types = soup.find('span', attrs={'id':'type-val'}).text.replace("\n",'').replace(" ",'').split(",")
        types = sorted(types)
        types.insert(0, b)
        type_of_id.append(types)
    except:
        error_links.append(b)

In [ ]:
error_links

In [ ]:
bug_links = []
for d, types in enumerate (id_type):
    if types[1] == 'Bug':
        bug_links.append(types[0])

print ("Number of bug id: " + str(len(bug_links)))
print (bug_links)

In [ ]:
with open ("bug_links.txt", mode="wt", encoding="utf-8") as myfile:
    myfile.write('\n'.join(bug_links))
print ("File bug_links.txt has been created")

In [ ]:
com_log = []
for no, bug_link in enumerate(bug_links):
    sys.stdout.write("\r%i " %(no+1) + "Extracting bug id: " + bug_link)
    sys.stdout.flush()
    git_cmd = "git log --oneline | grep -w '" + bug_link + "'"
    temp = (str(execute_command(git_cmd, repository)).replace("b'",'').replace('b"','').replace('(','',1).split("\\n"))[:-1]

    com_log.append(temp)

print ("\n")
for item in com_log:
    print(item)

In [ ]:
commits = []
for x in range(0,len(com_log)):
    com = []
    com.append(bug_links[x])
    for z in range(0,len(com_log[x])):
        temp = str(com_log[x][z].split()[0:1]).replace("['",'').replace("']",'')
        com.append(temp)
    commits.append(com)
print (commits)

# Bug-introducing change Commits

In [ ]:
bugintro_cid = []
for jj, kk in enumerate(commits):
    if (len(kk) > 1):
        bi_cid = kk[len(kk)-1:]
        bi_cid.insert(0,kk[0])
        bugintro_cid.append(bi_cid)
print (bugintro_cid)

In [ ]:
len(bugintro_cid)

# Candidate bug fix Commits

In [ ]:
bugfix_cid = []
for jj, kk in enumerate(commits):
    if (len(kk) > 2):
        bugfix_cid.append(kk[0:len(kk)-1])
print (bugfix_cid)

In [ ]:
len(bugfix_cid)

In [ ]:
bugintro = go.Bar(
    x = ['Bug-introducing change commits'],
    y = [len(bugintro_cid)],
    name = 'Bug-introducing change commits'
)

bugfix = go.Bar(
    x = ['Bug-fix commits'],
    y = [len(bugfix_cid)],
    name = 'Bug-fix commits'
)

data = [bugintro, bugfix]
layout = go.Layout(
    title = 'The number of Commits'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=False)

# Finding parent_id of commits

In [ ]:
bugintro_cid[0]

In [ ]:
parent_id = []
for v, w in enumerate(bugintro_cid):
    if w != []:
        git_cmd = "git log --pretty=%P -n1 " + w[1]
        temp = (str(execute_command(git_cmd, repository)).replace("b'",'').replace('(','',1).split("\\n"))[:-1]
        temps = [w,temp]
        newlist = list(chain.from_iterable(temps))

    parent_id.append(newlist)
    
parent_id

In [ ]:
with open(analyze_dir + "file-diff/parentid.csv", 'w') as parent:
    header = ['bug_id','bugintro_commitID','parent_id']
    writers = csv.writer(parent, delimiter=',')
    writers.writerow(header)
    for item in parent_id:
        writers.writerow(item)

In [ ]:
dfparent = pd.read_csv(analyze_dir + "file-diff/parentid.csv")
dfparent = dfparent[header]
dfparent

# Applying git diff without any algorithms

In [ ]:
parent_id[0][1]

In [ ]:
diffs = []
for v,w in enumerate(parent_id):
    sys.stdout.write('\rExtracting parent id for commits: %i' %(v+1) + ' out of %i' %(len(parent_id)))
    sys.stdout.flush()
    diff_cmd = "git diff " + w[2] + " " + w[1] + " --numstat"
    tmp = (str(execute_shell_command(w[0], w[1] , w[2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("\\n",',').replace("(+)",'').replace("(-)",'').replace("\\t",',').
               replace(")",'').replace("|",'').replace(" ",""))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace(",b'",',').replace(", ",'').replace("\'",'').replace(",,",''))
    diffs.append([str(tmp2)])
    
diffresult = []
for item in diffs:
    if item == ', ':
        del item
    else:
        diffresult.append(item)
print ('\n')
for xx in diffresult:
    print (xx)

In [ ]:
kl = []
res = []
for v in range(0,len(diffresult)):
    for w in range(0,len(diffresult[v])):
        kl = diffresult[v][w].split(',')
        res.append(kl)
res

In [ ]:
with open(analyze_dir + 'file-diff/diff-bugs.csv', 'w') as csvfile:
    header = ['bug_id','bugintro_commitID', 'parent_id', '#insertions','#deletions','filename','#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(res)):
        i = 3
        j = 4
        k = 5
        for x in range(0,int((len(res[m])-3)/3)):
            if (res[m][i]) == "-":
                pass
            else:
                line_changed = int(res[m][i]) + int(res[m][j])
                try:
                    pattern = re.search(('({\w.*=>|{=>)'), res[m][k])
                    pattern = pattern.groups()[0]
                    v = [res[m][0], res[m][1], res[m][2], res[m][i], res[m][j], res[m][k].replace(pattern,"").replace("}",""), str(line_changed)]
                except:             
                    v = [res[m][0], res[m][1], res[m][2], res[m][i], res[m][j], res[m][k], str(line_changed)]
                writers.writerow(v)
            i = i + 3
            j = j + 3
            k = k + 3

# Applying git diff -w --ignore-blank-lines

In [ ]:
diffsw = []
for v,w in enumerate(parent_id):
    sys.stdout.write('\rExtracting parent id for commits: %i' %(v+1) + ' out of %i' %(len(parent_id)))
    sys.stdout.flush()
    diff_cmd = "git diff -w --ignore-blank-lines " + w[2] + " " + w[1] + " --numstat"
    tmp = (str(execute_shell_command(w[0], w[1] , w[2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("\\n",',').replace("(+)",'').replace("(-)",'').replace("\\t",',').
               replace(")",'').replace("|",'').replace(" ",""))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace(",b'",',').replace(", ",'').replace("\'",'').replace(",,",''))
    diffsw.append([str(tmp2)])
    
diffwresult = []
for item in diffsw:
    if item == ', ':
        del item
    else:
        diffwresult.append(item)
diffwresult[1]

In [ ]:
klw = []
resw = []
for v in range(0,len(diffwresult)):
    for w in range(0,len(diffwresult[v])):
        klw = diffwresult[v][w].split(',')
        resw.append(klw)
#resw

In [ ]:
with open(analyze_dir + 'file-diff/diffw-bugs.csv', 'w') as csvfile:
    header = ['bug_id','bugintro_commitID', 'parent_id', '#insertions','#deletions','filename','#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(resw)):
        i = 3
        j = 4
        k = 5
        for x in range(0,int((len(resw[m])-3)/3)):
            if (resw[m][i]) == "-":
                pass
            else:
                line_changed = int(resw[m][i]) + int(resw[m][j])
                try:
                    pattern = re.search(('({\w.*=>|{=>)'), resw[m][k])
                    pattern = pattern.groups()[0]
                    v = [resw[m][0], resw[m][1], resw[m][2], resw[m][i], resw[m][j], resw[m][k].replace(pattern,"").replace("}",""), str(line_changed)]
                except:             
                    v = [resw[m][0], resw[m][1], resw[m][2], resw[m][i], resw[m][j], resw[m][k], str(line_changed)]
                writers.writerow(v)
            i = i + 3
            j = j + 3
            k = k + 3

# Applying git diff -w --ignore-blank-lines --diff-algorithm=myers

In [ ]:
diffmyers = []
for v,w in enumerate(parent_id):
    sys.stdout.write('\rExtracting parent id for commits: %i' %(v+1) + ' out of %i' %(len(parent_id)))
    sys.stdout.flush()
    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=myers " + w[2] + " " + w[1] + " --numstat"
    tmp = (str(execute_shell_command(w[0], w[1] , w[2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("\\n",',').replace("(+)",'').replace("(-)",'').replace("\\t",',').
               replace(")",'').replace("|",'').replace(" ",""))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace(",b'",',').replace(", ",'').replace("\'",'').replace(",,",''))
    diffmyers.append([str(tmp2)])
    
diffmyersresult = []
for item in diffmyers:
    if item == ', ':
        del item
    else:
        diffmyersresult.append(item)

In [ ]:
klmyers = []
resmyers = []
for v in range(0,len(diffmyersresult)):
    for w in range(0,len(diffmyersresult[v])):
        klmyers = diffmyersresult[v][w].split(',')
        resmyers.append(klmyers)
resmyers

In [ ]:
with open(analyze_dir + 'file-diff/diffmyers-bugs.csv', 'w') as csvfile:
    header = ['bug_id','bugintro_commitID', 'parent_id', '#insertions','#deletions','filename','#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(resmyers)):
        i = 3
        j = 4
        k = 5
        for x in range(0,int((len(resmyers[m])-3)/3)):
            if (resmyers[m][i]) == "-":
                pass
            else:
                line_changed = int(resmyers[m][i]) + int(resmyers[m][j])
                try:
                    pattern = re.search(('({\w.*=>|{=>)'), resmyers[m][k])
                    pattern = pattern.groups()[0]
                    v = [resmyers[m][0], resmyers[m][1], resmyers[m][2], resmyers[m][i], resmyers[m][j], resmyers[m][k].replace(pattern,"").replace("}",""), str(line_changed)]
                except:             
                    v = [resmyers[m][0], resmyers[m][1], resmyers[m][2], resmyers[m][i], resmyers[m][j], resmyers[m][k], str(line_changed)]
                writers.writerow(v)
            i = i + 3
            j = j + 3
            k = k + 3

# Applying -w --ignore-blank-lines --diff-algorithm=minimal

In [ ]:
diffmin = []
for v,w in enumerate(parent_id):
    sys.stdout.write('\rExtracting parent id for commits: %i' %(v+1) + ' out of %i' %(len(parent_id)))
    sys.stdout.flush()
    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=minimal " + w[2] + " " + w[1] + " --numstat"
    tmp = (str(execute_shell_command(w[0], w[1] , w[2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("\\n",',').replace("(+)",'').replace("(-)",'').replace("\\t",',').
               replace(")",'').replace("|",'').replace(" ",""))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace(",b'",',').replace(", ",'').replace("\'",'').replace(",,",''))
    diffmin.append([str(tmp2)])
    
diffminresult = []
for item in diffmin:
    if item == ', ':
        del item
    else:
        diffminresult.append(item)

In [ ]:
klmin = []
resmin = []
for v in range(0,len(diffminresult)):
    for w in range(0,len(diffminresult[v])):
        klmin = diffminresult[v][w].split(',')
        resmin.append(klmin)
#resmin

In [ ]:
with open(analyze_dir + 'file-diff/diffmin-bugs.csv', 'w') as csvfile:
    header = ['bug_id','bugintro_commitID', 'parent_id', '#insertions','#deletions','filename','#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(resmin)):
        i = 3
        j = 4
        k = 5
        for x in range(0,int((len(resmin[m])-3)/3)):
            if (resmin[m][i]) == "-":
                pass
            else:
                line_changed = int(resmin[m][i]) + int(resmin[m][j])
                try:
                    pattern = re.search(('({\w.*=>|{=>)'), resmin[m][k])
                    pattern = pattern.groups()[0]
                    v = [resmin[m][0], resmin[m][1], resmin[m][2], resmin[m][i], resmin[m][j], resmin[m][k].replace(pattern,"").replace("}",""), str(line_changed)]
                except:             
                    v = [resmin[m][0], resmin[m][1], resmin[m][2], resmin[m][i], resmin[m][j], resmin[m][k], str(line_changed)]
                writers.writerow(v)
            i = i + 3
            j = j + 3
            k = k + 3

# Applying -w --ignore-blank-lines --diff-algorithm=patience

In [ ]:
diffpat = []
for v,w in enumerate(parent_id):
    sys.stdout.write('\rExtracting parent id for commits: %i' %(v+1) + ' out of %i' %(len(parent_id)))
    sys.stdout.flush()
    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=patience " + w[2] + " " + w[1] + " --numstat"
    tmp = (str(execute_shell_command(w[0], w[1] , w[2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("\\n",',').replace("(+)",'').replace("(-)",'').replace("\\t",',').
               replace(")",'').replace("|",'').replace(" ",""))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace(",b'",',').replace(", ",'').replace("\'",'').replace(",,",''))
    diffpat.append([str(tmp2)])
    
diffpatresult = []
for item in diffpat:
    if item == ', ':
        del item
    else:
        diffpatresult.append(item)

In [ ]:
klpat = []
respat = []
for v in range(0,len(diffpatresult)):
    for w in range(0,len(diffpatresult[v])):
        klpat = diffpatresult[v][w].split(',')
        respat.append(klpat)
#respat

In [ ]:
with open(analyze_dir + 'file-diff/diffpat-bugs.csv', 'w') as csvfile:
    header = ['bug_id','bugintro_commitID', 'parent_id', '#insertions','#deletions','filename','#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(respat)):
        i = 3
        j = 4
        k = 5
        for x in range(0,int((len(respat[m])-3)/3)):
            if (resmin[m][i]) == "-":
                pass
            else:
                line_changed = int(respat[m][i]) + int(respat[m][j])
                try:
                    pattern = re.search(('({\w.*=>|{=>)'), respat[m][k])
                    pattern = pattern.groups()[0]
                    v = [respat[m][0], respat[m][1], respat[m][2], respat[m][i], respat[m][j], respat[m][k].replace(pattern,"").replace("}",""), str(line_changed)]
                except:             
                    v = [respat[m][0], respat[m][1], respat[m][2], respat[m][i], respat[m][j], respat[m][k], str(line_changed)]
                writers.writerow(v)
            i = i + 3
            j = j + 3
            k = k + 3

# Applying -w --ignore-blank-lines --diff-algorithm=histogram

In [ ]:
diffhist = []
for v,w in enumerate(parent_id):
    sys.stdout.write('\rExtracting parent id for commits: %i' %(v+1) + ' out of %i' %(len(parent_id)))
    sys.stdout.flush()
    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=histogram " + w[2] + " " + w[1] + " --numstat"
    tmp = (str(execute_shell_command(w[0], w[1] , w[2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("\\n",',').replace("(+)",'').replace("(-)",'').replace("\\t",',').
               replace(")",'').replace("|",'').replace(" ",""))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace(",b'",',').replace(", ",'').replace("\'",'').replace(",,",''))
    diffhist.append([str(tmp2)])
    
diffhistresult = []
for item in diffhist:
    if item == ', ':
        del item
    else:
        diffhistresult.append(item)
#diffhistresult

In [ ]:
klhist = []
reshist = []
for v in range(0,len(diffhistresult)):
    for w in range(0,len(diffhistresult[v])):
        klhist = diffhistresult[v][w].split(',')
        reshist.append(klhist)
#reshist

In [ ]:
with open(analyze_dir + 'file-diff/diffhist-bugs.csv', 'w') as csvfile:
    header = ['bug_id','bugintro_commitID', 'parent_id', '#insertions','#deletions','filename','#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(reshist)):
        i = 3
        j = 4
        k = 5
        for x in range(0,int((len(reshist[m])-3)/3)):
            if (reshist[m][i]) == "-":
                pass
            else:
                line_changed = int(reshist[m][i]) + int(reshist[m][j])
                try:
                    pattern = re.search(('({\w.*=>|{=>)'), reshist[m][k])
                    pattern = pattern.groups()[0]
                    v = [reshist[m][0], reshist[m][1], reshist[m][2], reshist[m][i], reshist[m][j], reshist[m][k].replace(pattern,"").replace("}",""), str(line_changed)]
                except:             
                    v = [reshist[m][0], reshist[m][1], reshist[m][2], reshist[m][i], reshist[m][j], reshist[m][k], str(line_changed)]
                writers.writerow(v)
            i = i + 3
            j = j + 3
            k = k + 3

# Removing files with no deletions (buglines)

In [ ]:
cols = ['bug_id','bugintro_commitID','parent_id','#insertions','#deletions','filename','#line_changed']

In [ ]:
dfhist = pd.read_csv(analyze_dir + 'file-diff/diffhist-bugs.csv')
dfmin = pd.read_csv(analyze_dir + 'file-diff/diffmin-bugs.csv')
dfmyers = pd.read_csv(analyze_dir + 'file-diff/diffmyers-bugs.csv')
dfpat = pd.read_csv(analyze_dir + 'file-diff/diffpat-bugs.csv')

In [ ]:
dfhist = dfhist[cols][dfhist['#deletions'] != 0]
dfhist.to_csv(analyze_dir + 'file-diff/diffhist-bugs.csv')
dfhist

In [ ]:
dfmin = dfmin[cols][dfmin['#deletions'] != 0]
dfmin.to_csv(analyze_dir + 'file-diff/diffmin-bugs.csv')
dfmin

In [ ]:
dfmyers = dfmyers[cols][dfmyers['#deletions'] != 0]
dfmyers.to_csv(analyze_dir + 'file-diff/diffmyers-bugs.csv')
dfmyers

In [ ]:
dfpat = dfpat[cols][dfpat['#deletions'] != 0]
dfpat.to_csv(analyze_dir + 'file-diff/diffpat-bugs.csv')
dfpat

# Visualizing the number of changes in different diff using graph

In [ ]:
dfhist = pd.read_csv(analyze_dir + 'file-diff/diffhist-bugs.csv')
dfhist = dfhist[header]
dfmin = pd.read_csv(analyze_dir + 'file-diff/diffmin-bugs.csv')
dfmin = dfmin[header]
dfmyers = pd.read_csv(analyze_dir + 'file-diff/diffmyers-bugs.csv')
dfmyers = dfmyers[header]
dfpat = pd.read_csv(analyze_dir + 'file-diff/diffpat-bugs.csv')
dfpat = dfpat[header]

In [ ]:
#sorting data based on filename
dfhist = dfhist.sort_values('filename')
dfmyers = dfmyers.sort_values('filename')
dfmin = dfmin.sort_values('filename')
dfpat = dfpat.sort_values('filename')

In [ ]:
#Merge dataframes bug-hist and bug-myers
datamerge1 = dfhist.merge(dfmyers, on=['bug_id','bugintro_commitID', 'parent_id', 'filename'], how="outer", suffixes=('-hist', '-myers'))
#Merge dataframes bug-min and bug-pat
datamerge2 = dfmin.merge(dfpat, on=['bug_id','bugintro_commitID', 'parent_id', 'filename'], how="outer", suffixes=('-min', '-pat'))
#Merge all dataframes
datamerge3 = datamerge1.merge(datamerge2, on=['bug_id','bugintro_commitID', 'parent_id', 'filename'], how="outer")

#Fill NAs with 0 and sort by filename
datamerge3.fillna(0, inplace=True)
datamerge3 = datamerge3.sort_values(by ='#deletions-hist').reset_index(drop=True)
colnames = ['bug_id','bugintro_commitID', 'parent_id','filename','#deletions-hist','#deletions-myers','#deletions-min','#deletions-pat']
datamerge3 = datamerge3[colnames]

#save to csv file
datamerge3.to_csv(analyze_dir + 'file-diff/diffbugs_file.csv')

In [ ]:
#Remove duplicates data
datamerge4 = datamerge3[datamerge3.iloc[:,-4:].diff(axis=1).fillna(0).any(1)]
datamerge4 = datamerge4.sort_values('#deletions-myers', ascending=False)

#save to csv file
datamerge4.to_csv(analyze_dir + 'file-diff/diffbugs_file_differentvalue.csv')

In [ ]:
#Show the csv file with different value of number of line changed
data = pd.read_csv(analyze_dir + 'file-diff/diffbugs_file_differentvalue.csv')
data = datamerge4[colnames]
data